In [7]:
import face_recognition
import cv2
import os
import numpy as np
from moviepy.editor import VideoFileClip

# Load known faces and their names
known_face_encodings = []
known_face_names = []

# Path to the directory containing subdirectories of images for each person
known_faces_dir = r'E:\jupyter_notebook\Music\FOLDER_FOR_FACE_RECOGNITION'

# Loop through each person in the known_faces_dir
for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    
    # Loop through each image of the person
    for image_name in os.listdir(person_dir):
        image_path = os.path.join(person_dir, image_name)
        
        # Load the image
        image = face_recognition.load_image_file(image_path)
        
        # Get the face encodings for the image
        encodings = face_recognition.face_encodings(image)
        
        # There might be no face or multiple faces in the image
        if len(encodings) == 1:
            # Use the first encoding and append it to the list
            known_face_encodings.append(encodings[0])
            known_face_names.append(person_name)

In [8]:
# Define paths
input_video = r"E:\new_downloads_default_folder\trump2.mp4"
temp_video = r"E:\trump2_temp11.mp4"
output_video = r"E:\trump2_out11.mp4"

# Open input video file
video_capture = cv2.VideoCapture(input_video)
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_writer = cv2.VideoWriter(temp_video, fourcc, fps, (frame_width, frame_height))

In [9]:
# Process each frame
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert frame to RGB (face_recognition uses RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Initialize an array for names of detected faces
    face_names = []

    for face_encoding in face_encodings:
        # Check if the face matches any known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Use the closest match to determine the name
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

    # Label the faces in the frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Draw a label with the name above the face
        cv2.rectangle(frame, (left, top - 35), (right, top), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, top - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting frame live
    cv2.imshow('Video2', frame)

    # Write processed frame to output video
    output_writer.write(frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video_capture.release()
output_writer.release()
cv2.destroyAllWindows()

# Adding the original audio back to the video using moviepy
original_clip = VideoFileClip(input_video)
processed_clip = VideoFileClip(temp_video)

# Combine video and audio
final_clip = processed_clip.set_audio(original_clip.audio)
final_clip.write_videofile(output_video, codec="libx264")

# Clean up temporary files
#os.remove(temp_video)

Moviepy - Building video E:\trump2_out11.mp4.
MoviePy - Writing audio in trump2_out11TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video E:\trump2_out11.mp4



Moviepy - Done !
Moviepy - video ready E:\trump2_out11.mp4
